In [1]:
import sys
import random
from datetime import datetime, timedelta

import mysql.connector
from mysql.connector import errorcode

from faker import Faker
from faker.providers import DynamicProvider

In [2]:
casetype_provider = DynamicProvider(
     provider_name="case_type",
     elements=["robbery", "murder", "other"],
)

casestatus_provider = DynamicProvider(
    provider_name="case_status",
    elements=["ongoing", "solved", "onhold"]
)

fake = Faker()
fake.add_provider(casetype_provider)
fake.add_provider(casestatus_provider)

In [3]:
# Function to generate fake data using Faker library
def generate_fake_data():
    
    data = {
        'employee': [],
        'department': [],
        'cases': [],
        'suspect': [],
        'committed_crime': [],
        'report': [],
        'investigation': []
    }

    # Generate department data
    # field officers, administrative workers and investigators.

    data['department'].append((fake.unique.uuid4(), "field_officer", "fieldofficer@erasmusmail.com" , fake.phone_number))
    data['department'].append((fake.unique.uuid4(), "administrative", "administrative@erasmusmail.com", fake.phone_number))
    data['department'].append((fake.unique.uuid4(), "investigator", "investigator@erasmusmail.com", fake.phone_number))
        
        
    # Generate employee data
    for i in range(30):
        employee_id = fake.unique.uuid4()
        citizen_id = fake.unique.ssn()
        department_id = random.choice(data['department'])[0]
        first_name = fake.first_name()
        last_name = fake.last_name()
        starting_date = fake.date_between(start_date='-10y', end_date='today')
        gender = random.choice(['Male', 'Female'])
        birth_date = fake.date_of_birth()
        email = first_name + last_name + "@erasmusmail.com"
        phone_number = fake.phone_number()
        address = fake.address()
        data['employee'].append((employee_id, citizen_id, department_id, first_name, last_name, starting_date, gender, birth_date, email, phone_number, address))

    # Generate cases data
    for i in range(100):
        case_id = fake.case_type() + fake.unique.uuid4()
        date_occurred = fake.date_time_between(start_date='-1y', end_date='today')
        case_description = fake.sentence(nb_words=10)
        location = fake.address()
        case_status = fake.casestatus()
        data['cases'].append((case_id, date_occurred, case_description, location, case_status))

    # Generate suspect data
    for i in range(50):
        citizen_id = fake.unique.ssn()
        suspect_name = fake.name()
        gender = random.choice(['Male', 'Female'])
        birth_date = fake.date_of_birth()
        address = fake.address()
        phone_number = fake.phone_number()
        data['suspect'].append((citizen_id, suspect_name, gender, birth_date, address, phone_number))

    # Generate committed crime data
    for case in data['cases']:
        case_id = case[0]
        citizen_id = random.choice(data['suspect'])[0]
        data['committed_crime'].append((case_id, citizen_id))

    # Generate report data
    for i in range(50):
        report_id = fake.unique.random_number(digits=6)
        reported_to = random.choice(data['employee'])[0]
        opened_case = random.choice(data['cases'])[0]
        reporter_name = fake.name()
        report_date = fake.date_time_this_year()
        reporter_citizen_id = fake.ssn()
        reporter_phone = fake.phone_number()
        data['report'].append((report_id, reported_to, opened_case, reporter_name, report_date, reporter_citizen_id, reporter_phone))

    # Generate investigation data
    for report in data['report']:
        employee_id = report[1]
        case_id = report[2]
        start_date = report[4]
        data['investigation'].append((employee_id, case_id, start_date))

    return data

In [4]:
def read_csv():
    

SyntaxError: incomplete input (4166213827.py, line 2)

In [ ]:
def connect_to_database():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="gurolberkay",
            password="password",
            database="bpstation"
        )
        return conn
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your username or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)

# Function to insert data into database
def insert_data(conn, cursor, table_name, data):
    try:
        if table_name == 'department':
            cursor.executemany(f"INSERT INTO {table_name} VALUES (%s, %s, %s, %s)", data)
        elif table_name == 'employee':
            cursor.executemany(f"INSERT INTO {table_name} VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", data)
        elif table_name == 'cases':
            cursor.executemany(f"INSERT INTO {table_name} VALUES (%s, %s, %s, %s, %s)", data)
        elif table_name == 'suspect':
            cursor.executemany(f"INSERT INTO {table_name} VALUES (%s, %s, %s, %s, %s, %s)", data)
        elif table_name == 'committed_crime':
            cursor.executemany(f"INSERT INTO {table_name} VALUES (%s, %s)", data)
        elif table_name == 'report':
            cursor.executemany(f"INSERT INTO {table_name} VALUES (%s, %s, %s, %s, %s, %s, %s)", data)
        elif table_name == 'investigation':
            cursor.executemany(f"INSERT INTO {table_name} VALUES (%s, %s, %s)", data)
        conn.commit()
    except mysql.connector.Error as err:
        print(f"Error inserting data into {table_name} table:", err)

In [ ]:
if __name__ == "__main__":

    filename = sys.argv[2]
    if sys.argv[1] == 'generate':
        # Generate data
        fake_data = generate_fake_data()
        # Write data to file
        write_csv(filename, fake_data)

    elif sys.argv[1] == 'populate':
        
        # Connect to database
        conn = connect_to_database()
        cursor = conn.cursor()
        
        # Read data
        fake_data = read_csv(filename)
        
        # Insert data into tables
        for table_name, data in fake_data.items():
            insert_data(conn, cursor, table_name, data)

        # Close cursor and connection
        cursor.close()
        conn.close()
    

    